In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path

In [26]:
NORMAL_FOLDER = "raw_self_data/normal"
DEFECT_FOLDER = "raw_self_data/defect"
SAVE_NORMAL_FOLDER = "data_train/normal"
SAVE_DEFECT_FOLDER = "data_train/defect"
SAMPLING_RATE = 9000
FRAME = 256
NOVERLAP = FRAME // 2 

In [ ]:
def create_and_save_spectrogram(signal_data, sampling_rate, save_path, nperseg, noverlap):
    """
    Создает спектрограмму из сигнала и сохраняет ее как изображение.
    
    Аргументы:
    signal_data (np.array): Входной одномерный сигнал (временной ряд).
    sampling_rate (int): Частота дискретизации сигнала в Гц.
    save_path (str): Полный путь для сохранения файла (включая имя файла, например, 'data/normal/spec_01.png').
    """

    frequencies, times, Zxx = signal.stft(signal_data, fs=sampling_rate, nperseg=nperseg, noverlap=noverlap)
    dpi = 100
    img_size_px = 256 
    figsize_inch = img_size_px / dpi
    
    plt.figure(figsize=(figsize_inch, figsize_inch))

    plt.pcolormesh(times, frequencies, np.log(np.abs(Zxx) + 1e-9), cmap='gray')
    
    plt.axis('off')
    plt.gca().set_position([0, 0, 1, 1])
    
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    plt.savefig(save_path, dpi=dpi)
    
    plt.close()

In [28]:
def read_data_raw(FILE_PATH_RAW):
    header = []
    data = []
    with open(FILE_PATH_RAW, 'r') as file:
        for line in file:
            line = line.strip()
            if line == "Data as Time Sequence:":
                break
            if line:  
                header.append(line)
        
        for line in file:
            line = line.strip()
            if line:  
                try:
                    value = float(line)
                    data.append(value)
                except ValueError:
                    continue  
    return data, header

In [31]:
for file in Path(NORMAL_FOLDER).iterdir():
    if file.is_file():
        data, header = read_data_raw(file)
        name = file.name.split(".")[:-1][0]
        save_path = SAVE_NORMAL_FOLDER + "/" + name + ".png"
        create_and_save_spectrogram(data, SAMPLING_RATE, save_path, FRAME, NOVERLAP)

In [32]:
for file in Path(DEFECT_FOLDER).iterdir():
    if file.is_file():
        data, header = read_data_raw(file)
        name = file.name.split(".")[:-1][0]
        save_path = SAVE_DEFECT_FOLDER + "/" + name + ".png"
        create_and_save_spectrogram(data, SAMPLING_RATE, save_path, FRAME, NOVERLAP)